In [1]:
# Basic Imports
import numpy as np
import pandas as pd
import os
import zipfile
import matplotlib.pyplot as plt


from tqdm import tqdm_notebook

# TensorFlow Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Embedding,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

%matplotlib inline

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip
    

--2023-04-24 13:22:48--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.32.144, 142.251.33.48, 142.251.116.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.32.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: './cats_and_dogs_filtered.zip'

./cats_and_dogs_fil 100%[===================>]  65.43M  13.5MB/s    in 5.1s    

2023-04-24 13:22:53 (12.8 MB/s) - './cats_and_dogs_filtered.zip' saved [68606236/68606236]



## Unzip the Dogs vs Cats Dataset

In [3]:
dataset_path = './cats_and_dogs_filtered.zip'

In [4]:
zip_object = zipfile.ZipFile(dataset_path,'r')

In [5]:
zip_object.extractall('./')
zip_object.close()

## Set up Dataset Paths

In [26]:
dataset_path_new = './cats_and_dogs_filtered/'

In [27]:
train_dir = os.path.join(dataset_path_new, 'train')
validation_dir = os.path.join(dataset_path_new, 'validation')

## Build the Model

### Load the pre-trained model (MobileNetV2)

In [8]:
img_shape = (128,128,3)

In [9]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')

Metal device set to: Apple M1 Pro


In [10]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 16, 16, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 16, 16, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 16, 16, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 16, 16, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 8, 8, 384)    0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 8, 8, 384)   3456        ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 8, 8, 384)   1536        ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 8, 8, 96)    384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 8, 8, 576)    55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 8, 8, 576)   2304        ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 8, 8, 576)    0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 4, 4, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 4, 4, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 4, 4, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 4, 4, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

### Freeze the Base Model
Must freeze the base model in order to train the model for a specific task. If model is not frozen, it will change the initial weights. Goal is to take learned features from the ImageNet and only train custom layers that are added on top of the base model. 

In [11]:
base_model.trainable = False

### Defining the custom head for our network
Custom Head is always designed for task being solved
1. Freeze base model
2. Check output size from base model that will assist in defining number of layers to add on top.

In [12]:
# Check output size
# Ouput size is (4x4x1280), too big for custom head
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

Couple of solutions to fix this:
- Flatten: Not suitable for this as the size is too large
- Global Average Pooling: Takes whole input instead of processing it in parts and takes the average

In [13]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [14]:
global_avg_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [15]:
# Output layer should have same amount of units as classes
prediction_layer = Dense(units=1, activation='sigmoid')(global_avg_layer)

### Defining the Model

In [16]:
# Combine the two neural networks
model = Model(inputs=base_model.input, outputs=prediction_layer)

In [17]:
base_model.input

<KerasTensor: shape=(None, 128, 128, 3) dtype=float32 (created by layer 'input_1')>

In [18]:
prediction_layer

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 16, 16, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 16, 16, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 16, 16, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 16, 16, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 8, 8, 384)    0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 8, 8, 384)   3456        ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 8, 8, 384)   1536        ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 8, 8, 96)    384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 8, 8, 576)    55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 8, 8, 576)   2304        ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 8, 8, 576)    0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 4, 4, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 4, 4, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 4, 4, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 4, 4, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

### Compile the Model
- optimizer
- loss
- metrics

**Because we are using a pre-trained model, we will have to slow down the learning rate. This is the reason the optimizer is set in this method instead of the traditional string method.**

In [20]:
model.compile(optimizer=tf.keras.optimizers.legacy.RMSprop(learning_rate=0.0001), loss= 'binary_crossentropy', metrics=['accuracy'] )

## Create Data Generators
To be done before training the data. Use Data Generators for preprocessing the data. <br>

**RESIZE IMAGES**<br>
Big pre-trained architectures support only certain input sizes.

In [21]:
# Create Data Generators
data_gen_train = ImageDataGenerator(rescale=1/255)
data_gen_val = ImageDataGenerator(rescale=1/255)

In [28]:
# Train Generator
train_generators = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size= 128, \
                                                      class_mode='binary')

Found 2000 images belonging to 2 classes.


In [30]:
# Validation Generator
val_generators = data_gen_val.flow_from_directory(validation_dir, target_size=(128,128), batch_size= 128, \
                                                      class_mode='binary')

Found 1000 images belonging to 2 classes.


### Train the Model
- When using data generators to preprocess the data, fit_generator() should be used.

In [31]:
model.fit_generator(train_generators, epochs=5, validation_data=val_generators)

/var/folders/86/j98_vzjs5dg0v5262tfx5yhw0000gn/T/ipykernel_22518/3483112282.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generators, epochs=5, validation_data=val_generators)


Epoch 1/5


2023-04-24 13:41:57.640613: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


16/16 [==============================] - 9s 342ms/step - loss: 0.8226 - accuracy: 0.4700 - val_loss: 0.7669 - val_accuracy: 0.5140
Epoch 2/5
16/16 [==============================] - 4s 242ms/step - loss: 0.6933 - accuracy: 0.5870 - val_loss: 0.6670 - val_accuracy: 0.6160
Epoch 3/5
16/16 [==============================] - 4s 235ms/step - loss: 0.6032 - accuracy: 0.6880 - val_loss: 0.5851 - val_accuracy: 0.7050
Epoch 4/5
16/16 [==============================] - 4s 240ms/step - loss: 0.5266 - accuracy: 0.7465 - val_loss: 0.5145 - val_accuracy: 0.7620
Epoch 5/5
16/16 [==============================] - 4s 236ms/step - loss: 0.4622 - accuracy: 0.8090 - val_loss: 0.4539 - val_accuracy: 0.8030


### Transfer Learning Model Evaluation

In [33]:
valid_loss, valid_acc = model.evaluate_generator(val_generators)

/var/folders/86/j98_vzjs5dg0v5262tfx5yhw0000gn/T/ipykernel_22518/3438080458.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_acc = model.evaluate_generator(val_generators)


In [34]:
print (f' Valid Loss: {valid_loss}')
print (f' Valid Accuracy: {valid_acc}')

 Valid Loss: 0.45390355587005615
 Valid Accuracy: 0.8029999732971191


## Fine Tuning
A few pointers:
- **DO NOT** use Fine Tuning on the whole network. **Only** on a few top layers are enough. In most cases, they are more specialized. The main goal of Fine Tuning is to incorporate that specific part of the network for our custom/new dataset.


- Start Fine Tuning **AFTER** you have finished with the transfer learning step. If Fine Tuning is done prior to this, gradients will be different between our custom head layer and a few unfrozen layers from the base model.

### Unfreeze a few top layers from the model

In [35]:
# Unfreeze the model
base_model.trainable = True

In [38]:
# Check number of layers in the model
print(f'Number of layers in the base model: {len(base_model.layers)}')

Number of layers in the base model: 154


In [39]:
# Start fine tuning at layer 100
fine_tune_at = 100

In [40]:
# Iterate through each layer and freeze all layers up to 100.
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compile the Model for Fine Tuning

In [47]:
model.compile(optimizer=tf.keras.optimizers.legacy.RMSprop(learning_rate=0.0001), loss='binary_crossentropy',
                                                   metrics=['accuracy'])

### Train the Fine Tuned Model

In [48]:
model.fit_generator(train_generators, epochs=5, validation_data=val_generators)

/var/folders/86/j98_vzjs5dg0v5262tfx5yhw0000gn/T/ipykernel_22518/3483112282.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generators, epochs=5, validation_data=val_generators)


Epoch 1/5
16/16 [==============================] - 13s 442ms/step - loss: 0.1992 - accuracy: 0.9255 - val_loss: 0.1051 - val_accuracy: 0.9640
Epoch 2/5
16/16 [==============================] - 4s 267ms/step - loss: 0.0334 - accuracy: 0.9940 - val_loss: 0.1082 - val_accuracy: 0.9670
Epoch 3/5
16/16 [==============================] - 4s 237ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.1288 - val_accuracy: 0.9680
Epoch 4/5
16/16 [==============================] - 4s 234ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.1239 - val_accuracy: 0.9660
Epoch 5/5
16/16 [==============================] - 4s 235ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.1414 - val_accuracy: 0.9680


In [51]:
val_loss, val_acc = model.evaluate_generator(val_generators)

/var/folders/86/j98_vzjs5dg0v5262tfx5yhw0000gn/T/ipykernel_22518/2391827049.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  val_loss, val_acc = model.evaluate_generator(val_generators)


In [54]:
print (f' Valid Loss: {val_loss}')
print (f' Valid Accuracy after Fine Tuning: {val_acc}')

 Valid Loss: 0.14137551188468933
 Valid Accuracy after Fine Tuning: 0.9679999947547913
